In [ ]:
# setup
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import (
    ColumnDataSource, Range1d, DataRange1d, DatetimeAxis,
    TickFormatter, DatetimeTickFormatter, FuncTickFormatter,
    Grid, Legend, Plot, BoxAnnotation, Span, CustomJS, Rect, Circle, Line,
    HoverTool, BoxZoomTool, PanTool, WheelZoomTool,
    WMTSTileSource, LabelSet
    )
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn, Div

import numpy as np
import sys
import os

sys.path.append(os.path.join(os.getcwd(), 'plot_app'))
from pyulog import *
from pyulog.px4 import *
from plotting import *
from config import *
from notebook_helper import *

output_notebook()

In [ ]:
file_name = '' # TODO: fill in file name
ulog = ULog(file_name)
data = ulog.data_list
px4_ulog = PX4ULog(ulog)
px4_ulog.add_roll_pitch_yaw()
use_downsample = False

print("message names: {:}".format(sorted([d.name for d in data])))
print_ulog_info(ulog)

In [ ]:
use_downsample = True # you may want to activate this for large logs (But you will not see all samples when zooming in)

In [ ]:
# load a dataset to inspect the field names and types
sensor_data = [ elem for elem in data if elem.name == 'vehicle_local_position' and elem.multi_id == 0][0]
types_list = [(f.type_str, f.field_name) for f in sensor_data.field_data]
for ftype, fname in types_list: print("{:10s} {:}".format(ftype, fname))

In [ ]:
plot = plot_map(data, plot_config) # gps map
if plot != None: show(plot)

In [ ]:
# plot raw acceleration sensor data
data_plot = DataPlot(data, plot_config, 'sensor_combined',
        y_axis_label = '[m/s^2]', title = 'Raw Acceleration', plot_height = 'normal')
data_plot.add_graph(['accelerometer_m_s2[0]', 'accelerometer_m_s2[1]', 'accelerometer_m_s2[2]'],
                    colors3, ['x', 'y', 'z'], use_downsample=use_downsample)
show(data_plot.finalize())

In [ ]:
# a more complex plot with multiple datasets
data_plot = DataPlot(data, plot_config, 'sensor_combined', y_start=0, title = 'Thrust and Magnetic Field',
                     plot_height='normal')
data_plot.add_graph([lambda data: ('len_mag', 
    np.sqrt(data['magnetometer_ga[0]']**2 + data['magnetometer_ga[1]']**2 + data['magnetometer_ga[2]']**2))],
    colors2[0:1], ['Norm of Magnetic Field'], use_downsample=use_downsample)

data_plot.change_dataset('actuator_controls')
data_plot.add_graph([lambda data: ('thrust', data['control[3]'])], colors2[1:2], ['Thrust'],
                    use_downsample=use_downsample)

show(data_plot.finalize())

In [ ]:
# plot low-pass filtered raw acceleration sensor data
from scipy.signal import butter, lfilter
cur_data = ulog.get_dataset('sensor_combined').data
t = cur_data['timestamp']

fs = len(t) / ((t[-1]-t[0])/1e6) # sample rate [Hz]
cutoff = 10 # cutoff freq [Hz]
order = 5
B, A = butter(order, cutoff / (fs / 2), btype='low') # Butterworth low-pass
filtered_signal = lfilter(B, A, cur_data['accelerometer_m_s2[0]'])

p = figure(plot_width = 800, active_scroll='wheel_zoom')
p.line(t, cur_data['accelerometer_m_s2[0]'], color='red', alpha = 0.5)
p.line(t, filtered_signal, color='blue', alpha = 0.8)
show(p)